In [3]:
import polars as pl
pl.Config().set_tbl_rows(1000)
pl.Config().set_fmt_str_lengths(50)
pl.Config().set_fmt_table_cell_list_len(20)
from datetime import datetime, time, timedelta
import sys
sys.path.append("..")

from libs.BusNetwork import BusNetwork

In [4]:
routes_df = pl.read_parquet('../static/routes.parquet')
assert routes_df.filter(pl.col('name').is_duplicated()).shape[0] == 0, 'Duplicate route names found'
estaciones_df = pl.read_csv('../static/_dim_estaciones.csv')
estaciones = {}
for row in estaciones_df.to_dicts():
    estaciones[row['id']] = row['name']

In [5]:
dir = 'NS'
# time_now = datetime.now().time()
time_now = time(7,0)
# day_now = datetime.now().weekday()
day_now = 2
inicio = 10
fin = 34
# inicio = 1
# fin = 7


routes_df = routes_df.filter(
    (pl.col('schedule').list.first()<time_now)
    &(pl.col('schedule').list.last()>time_now)
    &(pl.col('days').list.contains(day_now))
    &(pl.col('direction')==dir)
    )

In [6]:
network = BusNetwork()
rutas = {}

for row in routes_df.to_dicts():
    network.add_route(row['name'], row['stops'])
    rutas[row['name']] = row['commercial_name']

In [7]:
def route_stats(routes, path):
    unique_buses= []
    stops = []
    aux_current = None
    for x,y in enumerate(routes):
        if y != aux_current:
            unique_buses.append(y)
            if x != 0:
                stops.append(path[x-1])
                # stops.append(path[x])

            aux_current = y
    stops.append(path[-1])
    return unique_buses, stops

In [8]:


print('Inicio: ', estaciones[inicio])
print('Fin: ', estaciones[fin])
path, routes = network.find_shortest_path(inicio, fin)
print('Path: ', path)
print('Routes: ', routes)

Inicio:  Tomas Valle
Fin:  28 de Julio
Path:  [23, 27, 32, 34]
Routes:  ['Expreso7|NS', 'Expreso7|NS', 'Expreso2|NS|1', 'Expreso2|NS|1']


In [9]:
path, routes = network.find_shortest_path(inicio, fin)
unique_buses, stops = route_stats(routes, path)

print(f"Para ir de {estaciones[inicio]} hasta {estaciones[fin]} tienes que tomar:")

for i in range(len(unique_buses)):
    if i == 0:
        print(f"La ruta {rutas[unique_buses[i]]} desde {estaciones[inicio]} hasta {estaciones[stops[i]]}")
    elif i == len(unique_buses)-1:
        print(f"La ruta {rutas[unique_buses[i]]} desde {estaciones[stops[i-1]]} hasta {estaciones[fin]}")
    else:
        print(f"La ruta {rutas[unique_buses[i]]} desde {estaciones[stops[i-1]]} hasta {estaciones[stops[i]]}")

Para ir de Tomas Valle hasta 28 de Julio tienes que tomar:
La ruta Expreso7 desde Tomas Valle hasta Javier Prado
La ruta Expreso2 desde Javier Prado hasta 28 de Julio


In [10]:
print(f"Para ir de {inicio} hasta {fin} tienes que tomar:")

for i in range(len(unique_buses)):
    if i == 0:
        print(f"La ruta {rutas[unique_buses[i]]} desde {inicio} hasta {stops[i]}")
    elif i == len(unique_buses)-1:
        print(f"La ruta {rutas[unique_buses[i]]} desde {stops[i-1]} hasta {fin}")
    else:
        print(f"La ruta {rutas[unique_buses[i]]} desde {stops[i-1]} hasta {stops[i]}")

Para ir de 10 hasta 34 tienes que tomar:
La ruta Expreso7 desde 10 hasta 27
La ruta Expreso2 desde 27 hasta 34


In [11]:
routes_df

commercial_name,name,direction,days,shift,schedule,stops
str,str,str,list[i64],str,list[time],list[i64]
"""A""","""A1""","""NS""","[0, 1, 2, 3, 4, 5]",null,"[05:00:00, 23:00:00]","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 20, 21, 22, 23]"
"""C""","""C1""","""NS""","[0, 1, 2, 3, 4, 5]",null,"[05:00:00, 23:00:00]","[16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, … 42]"
"""D""","""D1""","""NS""","[0, 1, 2, 3, 4, 5]",null,"[05:00:00, 10:30:00]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 23]"
"""Expreso1""","""Expreso1|NS|M|1""","""NS""","[0, 1, 2, 3, 4]","""M""","[05:30:00, 09:30:00]","[23, 24, 27, 28, 31, 34, 36, 42]"
"""Expreso2""","""Expreso2|NS|1""","""NS""","[0, 1, 2, 3, 4]","""M""","[05:00:00, 09:00:00]","[5, 26, 27, 32, 34]"
"""Expreso3""","""Expreso3|NS1""","""NS""","[0, 1, 2, 3, 4]","""M""","[05:30:00, 09:00:00]","[5, 31, 33, 34]"
"""Expreso6""","""Expreso6|NS""","""NS""","[0, 1, 2, 3, 4]","""M""","[05:30:00, 10:00:00]","[6, 8, 23, 27, 28, 31, 33]"
"""Expreso7""","""Expreso7|NS""","""NS""","[0, 1, 2, 3, 4]","""M""","[05:30:00, 09:00:00]","[10, 23, 27, 28, 31]"
"""Expreso9""","""Expreso9|NS""","""NS""","[0, 1, 2, 3, 4]","""M""","[05:30:00, 09:00:00]","[13, 15, 26, 28, 31, 33]"


In [12]:
network.graph

#Now print it sorted by the key

for key in sorted(network.graph.keys()):
    print("%s: %s" % (key, network.graph[key]))

1: {2: [('D1', 1), ('Expreso11|NS', 1)]}
2: {3: [('D1', 1), ('Expreso11|NS', 1)]}
3: {4: [('D1', 1), ('Expreso11|NS', 1)]}
4: {5: [('D1', 1)], 7: [('Expreso11|NS', 1.0)]}
5: {6: [('A1', 1), ('D1', 1)], 26: [('Expreso2|NS|1', 1.4)], 31: [('Expreso3|NS1', 1.7333333333333334)], 15: [('Expreso10|NS', 1.6666666666666667)], 28: [('SX|NS', 1.5333333333333334)], 17: [('SNX|NSM', 1.5)]}
6: {7: [('A1', 1), ('D1', 1)], 8: [('Expreso6|NS', 0.6666666666666666)]}
7: {8: [('A1', 1), ('D1', 1)], 23: [('Expreso11|NS', 2.0)]}
8: {9: [('A1', 1), ('D1', 1)], 23: [('Expreso6|NS', 1.875)]}
9: {10: [('A1', 1), ('D1', 1)]}
10: {11: [('A1', 1), ('D1', 1)], 23: [('Expreso7|NS', 1.625)]}
11: {12: [('A1', 1), ('D1', 1)]}
12: {13: [('A1', 1), ('D1', 1)]}
13: {14: [('A1', 1), ('D1', 1)], 15: [('Expreso9|NS', 0.6666666666666666)]}
14: {15: [('A1', 1), ('D1', 1)]}
15: {16: [('A1', 1), ('Expreso10|NS', 1)], 17: [('D1', 0.6666666666666666)], 26: [('Expreso9|NS', 1.375)]}
16: {20: [('A1', 1.0), ('C1', 1.0), ('Expreso10|